### 特征工程
    IV值
        全称是Information Value，中文意思是信息价值，或者信息量。假设在一个分类问题中，目标变量的类别有两类：Y1，Y2。对于一个待预测的个体A，要判断A属于Y1还是Y2，我们是需要一定的信息的，假设这个信息总量是I，而这些所需要的信息，就蕴含在所有的自变量C1，C2，C3，……，Cn中，那么，对于其中的一个变量Ci来说，其蕴含的信息越多，那么它对于判断A属于Y1还是Y2的贡献就越大，Ci的信息价值就越大，Ci的IV就越大，它就越应该进入到入模变量列表中。
        计算：
        IV的计算是以WOE（证据权重）为基础的。

    
    随机森林
        随机森林是以决策树为基学习器的集成学习算法，在分类和回归上效果很好

### IV值方法

In [ ]:
def CalcIV(Xvar,Yvar):
    N_0=np.sum(Yvar==0)
    N_1=np.sum(Yvar==1)
    N_0_group=np.zeros(np.unique(Xvar).shape)
    
    N_1_group=np.zeros(np.unique(Xvar).shape)
    for i in range(len(np.unique(Xvar))):
        N_0_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==0)].count()
        N_1_group[i] = Yvar[(Xvar==np.unique(Xvar)[i])&(Yvar==1)].count()
    iv = np.sum((N_0_group/N_0-N_1_group/N_1)*np.log((N_0_group/N_0)/(N_1_group/N_1)))
    if iv>=1.0:## 处理极端值
        iv=1
    return iv

def caliv_batch(df,Yvar):
    ivlist=[]
    for col in df.columns:
        iv=CalcIV(df[col],Yvar)
        ivlist.append(iv)
    names=list(df.columns)
    iv_df=pd.DataFrame({'Var':names,'Iv':ivlist},columns=['Var','Iv'])

    return iv_df,ivlist
im_iv, ivl = caliv_batch(datafinal,data_train)


In [ ]:
#再数据处理：
threshold = 0.02
threshold2 = 0.6
data_index=[]
for i in range(len(ivl)):
    if (im_iv['Iv'][i]< threshold)|(im_iv['Iv'][i] > threshold2):
        data_index.append(im_iv['Var'][i])
datafinal.drop(data_index,axis=1,inplace=True)


### 随机森林：

In [ ]:

from sklearn.ensemble import RandomForestClassifier
feat_lables = datafinal.columns
forest = RandomForestClassifier(n_estimators=10000, random_state=0,n_jobs=1)
forest.fit(datafinal, data_train)
importance = forest.feature_importances_
imp_result = np.argsort(importance)[::-1]

for i in range(datafinal.shape[1]):
    print("%2d. %-*s %f"%(i+1, 30, feat_lables[i], importance[imp_result[i]]))


In [ ]:
#筛选：
threshold = 0.01
data_index = list(datafinal.columns[ importance < threshold])
datafinal.drop(data_index,axis=1,inplace=True)
datafinal.shape
